In [1]:
import pandas as pd
import numpy as np
import os
import csv
import argparse
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
import timeit
from memory_profiler import memory_usage

In [8]:
def load_csv(path):
    data = []
    with open(os.path.join(path), 'r') as file:
        reader = csv.reader(file, delimiter=',')
        data = list(reader)
        # data = list(filter(None, data))

    for row in data:
        while('' in row):
            row.remove('')

    # Transform the input dataset into a one-hot encoded NumPy boolean array
    # te = TransactionEncoder()
    # data = pd.DataFrame(te.fit(data).transform(data), columns=te.columns_)

    return data

In [16]:
ratings_df = pd.read_csv('datasets/ratings_small.csv')
movies_df = pd.read_csv('datasets/movies_metadata.csv')
title_mask = movies_df['title'].isna()
movies_df = movies_df.loc[title_mask == False]
movies_df = movies_df.astype({'id': 'int64'})
df = pd.merge(ratings_df, movies_df[['id', 'title']], left_on='movieId', right_on='id')
df.drop(['timestamp', 'id'], axis=1, inplace=True)
df = df.drop_duplicates(['userId','title'])
df_pivot = df.pivot(index='userId', columns='title', values='rating').fillna(0)
df_pivot = df_pivot.astype('int64')
def encode_ratings(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

df_pivot = df_pivot.applymap(encode_ratings)

C:\Users\lerus\AppData\Local\Temp\ipykernel_18232\1666700984.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('datasets/movies_metadata.csv')


In [20]:
def find_frequent_itemset(data, minsup, mode="None"):
    # by passing minsup we mean the number of times in total number
    # of transactions the item should be present
    frequent_itemsets = apriori(data, min_support=minsup, use_colnames=True)

    match mode:
        case "descending-support":
            return frequent_itemsets.sort_values(by=['support'], ascending=False)
        case "itemsets":
            return frequent_itemsets.sort_values(by=['itemsets'], ascending=True)
        case _:
            return frequent_itemsets

In [22]:
import timeit

minsup = 0.1
num_runs = 5
duration = timeit.Timer("find_frequent_itemset(df, minsup)", globals=locals()).timeit(number = num_runs)
avg_duration = duration/num_runs

print(find_frequent_itemset(df, minsup).size)

print(f'On average it took {avg_duration} seconds')

ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value 1371

In [19]:
frequent_itemset = apriori(df_pivot, min_support=0.07, use_colnames=True)
frequent_itemset.head()

,support,itemsets
0,0.129657,"(20,000 Leagues Under the Sea)"
1,0.129657,(2001: A Space Odyssey)
2,0.298063,(48 Hrs.)
3,0.292101,(5 Card Stud)
4,0.093890,(A Brief History of Time)


In [9]:
df = load_csv("datasets/movies_metadata.csv")

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 862: character maps to <undefined>

In [3]:
def find_frequent_itemset(data, minsup, mode="None"):
    # by passing minsup we mean the number of times in total number
    # of transactions the item should be present
    frequent_itemsets = apriori(data, min_support=minsup, use_colnames=True)

    match mode:
        case "descending-support":
            return frequent_itemsets.sort_values(by=['support'], ascending=False)
        case "itemsets":
            return frequent_itemsets.sort_values(by=['itemsets'], ascending=True)
        case _:
            return frequent_itemsets

In [4]:
df = load_csv("datasets/movies_metadata.csv")
frequent_itemset = find_frequent_itemset(df, 0.1)
print(frequent_itemset)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 862: character maps to <undefined>